In [61]:
# Top N问题，查看用户是否会给某部电影评分
# dataset： MovieLens

def SplitData(data,M,k,seed):
    '''
    Split data
    M : experimental times(M-fold)
    k: the test section
    seed: random seed
    '''
    test = []
    train = []
    random.seed(seed)
    for user, item in data:
        if random.randint(0,M) == k:
            test.append([user,item])
        else:
            train.append([user,item])
    return train,test

# 评价指标： 召回率，精确度，覆盖度,新颖度

def Recall(train, test, N):
    '''
    N : top_N
    '''
    hit = 0
    alls = 0
    for user in train.keys():
        tu = test[user] # 真实的样本
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        alls += len(tu)
    return hit / float(alls)

def Precision(train, test, N):
    hit = 0
    alls = 0
    for user in train.keys():
        tu = test[user]
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        alls += len(rank)
    return hit / float(alls) # /(alls*1.0)
    
def Coverage(train, test, N):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item in train[user]:
            all_items.add(item)
        rank = GetRecommendation(user,N)
        for item, pui in rank:
            recommend_items.add(item)
    return len(recommend_items)/(len(all_items)*1.0)

def Popularity(train, test, N):
    item_popularity = dict()
    for user,items in train.items():
        for item in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in train.keys():
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            ret += math.log(1+item_popularity[item])
            n += 1
    ret = ret/(n*1.0)
    return ret

def GiniIndex(p):
    """
    p物品流行度
    """
    j = 1
    n = len(p)
    G = 0
    for item, weight in sorted(p.items(), key=itemgetter(1)):
        G += (2*j - n - 1)*weights
        j += 1 # 自己添加的
    return G / float(n-1)

Jaccard: $ w_{uv} = \frac{N(u)\cap N(v)}{N(u)\cup N(v)}$  
cosine: $w_{uv} = \frac{N(u)\cap N(v)}{\sqrt{|N(u)||N(v)|}}$

In [64]:
# cosine
import math
def UserSimilarityold(train):
    '''
    这种算法时间复杂度太大，所以一般不用
    '''
    W = dict()
    for u in train.keys():
        for v in train.keys():
            if u == v:
                continue
            w[u][v] = len(train[u]&train[v])
            w[u][v] /= math.sqrt(len(train[u]) * len(train[v] * 1.0))
    return w

def UserSimilarity(train):
    """
    这个相似度实现的是，首先建立物品到用户的排序表；
    如果说u，v客户同属于物品到用户排序表种第K个的话，C[u][k] = K
    """
    # build inverse table for item_users(建立物品到用户的倒排表)
    item_users = {}
    for u, items in train.items():
        #for i in items.keys():
        for i in items:
            if i not in item_users.keys():
                item_users[i] = set()
            item_users[i].add(u)
            
    # calculate co-rated items between users
    C = {}
    N = {}
    for i, users in item_users.items():
        for u in users:
            if u not in N.keys():
                N[u] = 0
            if u not in C.keys():
                C[u] = {}
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                if v not in C[u].keys():
                    C[u][v] = 0
                C[u][v] += 1
    
    # calculate finial similarity matrix W
    w = {}
    for u, related_users in C.items():
        for v, cuv in related_users.items():
            if u not in w.keys():
                w[u] = {}
            w[u][v] = cuv/math.sqrt(N[u]*N[v])
    return w,N

In [66]:
train = {'A':['a','b','d'],'B':['a','c'],'C':['b','e'],'D':['c','d','e']}

W,N = UserSimilarity(train)

W['A']['B'] + W['A']['D']
N

{'A': 3, 'B': 2, 'C': 2, 'D': 3}

In [59]:
def Recommend(user,train, W, K):
    rank = {}
    rvi = 1# 这个是用户对物品的兴趣度，可以自定义取值，
    #当train是嵌套字典的时候，物品后面可以跟rvi值，train[user].keys()也就是用户一般浏览的物品
    interacted_items = train[user]
    # print(interacted_items)
    similarity = sorted(W[user].items(),key = lambda x:x[1], reverse = True)[0:K]
    for v, wuv in similarity: # 选取用户近似度排名前K名的
        #print(v,wuv)
        for i in train[v]: # 相似用户的物品
            if i in interacted_items: # 过滤用户和相似用户都已经有的物品
                # filter items user interacted before
                continue
            if i not in rank.keys():
                rank[i] = 0
            #print('rank',i,rank[i])
            #print(wuv,rvi)
            rank[i] += wuv*rvi
            #print('rank',i,rank[i])
    return rank
    

In [60]:
Recommend('A',train,W,3)

{'c': 0.7415816237971964, 'e': 0.7415816237971964}

改进余弦相似度：
对于余弦相似度的分子做一个惩罚项，也就是除以log(两个用户共同商品数量+1)
 $$w_{uv} = \frac{\sum_{i \in N(u)\cap N(v)}\frac{1}{log(1+|N(i)|)}}{\sqrt{|N(u)||N(v)|}}$$  
 N(i)是指对物品i有过惯量的用户的集合

In [68]:
def UserSimilarityChange(train):
    # 建立物品用户表：
    item_users = {}
    for u, items in train.items():
        for item in items:
            if item not in item_users.keys():
                item_users[key] = set()
            item_users[key].add(u)
            
    # 计算用户之间的相似性：
    C = {}
    N = {}
    for item, users in item_users.items:
        for u in users:
            if u not in N.keys():
                N[u] = 0
            N[u] += 1 # 用户u有过相关的物品数目
            if u not in C.keys():
                C[u] = {}
            for v in users:
                if u == v:
                    continue
                if v not in C[u].keys():
                    C[u][v] = 0
                C[u][v] += 1 / math.log(1+len(users)) # item_users[item] = user
    
    W = {}
    for u, related_users in C.items():
        if u not in W.keys():
            W[u] = {}
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N(u)*N(v))
    return W

### 以下是基于物品的协同过滤
基于物品的协同过率不利用物品内容的属性计算物品之间的相似度，主要通过分析用户的行为记录计算物品之间的相似度。

步骤：
1、计算物品之间的相似度。
2、根据物品的相似度和用户的历史行为给用户生成推荐列表。

定义喜欢i物品的用户喜欢j物品的概率（相似度）：$ w_{ij} = \frac{|N(i)\cap N(j)|}{|N(i)|}$  
|N(i)|表明喜欢i物品的用户数目  
但是如果|N(j)|属于热门商品，多数用户都喜欢，那么所有的热门商品的$w_{ij}$都接近1，所以热门物品与其他物品的相似度都接近1，那么无法挖掘冷门商品。  

修改相似度计算：
$ w_{ij} = \frac{|N(i)\cap N(j)|}{\sqrt{|N(i)||N(j)|}}$ 

In [70]:
def ItemSimilarity(train):
    # 计算用户和物品之间的协同过滤
    C = {}
    N = {}
    for user,items in train.items():
        for item in items:
            if not item in N.keys():
                N[item] = 0
            if not item in C.keys():
                C[item] = {}
            N[item] += 1
            for itemv in items:
                if item == itemv:
                    continue
                if itemv not in C[item].keys():
                    C[item][itemv] = 0
                C[item][itemv] += 1
    
    # 计算相似度
    W = {}
    for item, related_items in C.items():
        if item not in W.keys():
            W[item] = {}
            for itemv, cuv in related_items.items():
                W[item][item] = cuv / math.sqrt(N[item]*N[itemv])
    
    return W,N,C
            

In [71]:
W,N,C = ItemSimilarity(train)

In [72]:
C

{'a': {'b': 1, 'c': 1, 'd': 1},
 'b': {'a': 1, 'd': 1, 'e': 1},
 'c': {'a': 1, 'd': 1, 'e': 1},
 'd': {'a': 1, 'b': 1, 'c': 1, 'e': 1},
 'e': {'b': 1, 'c': 1, 'd': 1}}

In [73]:
def ItemRecommendation(user,train, W, K):
    rank = {}
    ru = train[user] # 真实用户喜欢的item
    pi = 1
    for i in ru:
        for j, wj in sorted(W[i].items(),key = lambda x: x[1],reverse=True)[0:K]:
            if j in ru:
                continue
            if j not in rank.keys():
                rank[j] = 0
            rank[j] += wj*pi
    return rank
            

相似度值修改：  
因为如果说一个异常的用户，让所有物品的80%的物品都有关联，那么这些物品就都有了相似度，但是其实这个用户的贡献值是我们不需要的。  
所以定义Inverse User Frequence:  

 $$w_{uv} = \frac{\sum_{u \in N(i)\cap N(j)}\frac{1}{log(1+|N(u)|)}}{\sqrt{|N(i)||N(j)|}}$$  
 |N(u)|是与用户相关的物品数目

In [74]:
def ItemSimilarityChange(train):
    # 计算用户和物品之间的协同过滤
    C = {}
    N = {}
    for user,items in train.items():
        for item in items:
            if not item in N.keys():
                N[item] = 0
            if not item in C.keys():
                C[item] = {}
            N[item] += 1
            for itemv in items:
                if item == itemv:
                    continue
                if itemv not in C[item].keys():
                    C[item][itemv] = 0
                C[item][itemv] += 1 / math.log(1+len(items))
    
    # 计算相似度
    W = {}
    for item, related_items in C.items():
        if item not in W.keys():
            W[item] = {}
            for itemv, cuv in related_items.items():
                W[item][item] = cuv / math.sqrt(N[item]*N[itemv])
    
    return W,N,C

对用户之间的相似度做归一化处理也可以提高推荐系统的多样性。
归一化：  
$$w_{ij}^{'} = \frac{w_{ij}}{max_{j} w_{ij}}$$
这样可以对每一个同类物品的相似度最大置为1
例如，如果说A类商品的相似度是0.5，B类商品的相似度是0.6，AB类商品之间的相似度是0.2。  
如果说用户喜欢5个A类和5个B类的商品，ItemCF推荐的为B，因为相似度大。归一化之后，A，B相似度都是1，那么推荐系统会均匀推荐A和B，提高了推荐系统的多样性。

### UserCF 和 ItemCF的差别  

UserCF推荐结果注重反映和用户兴趣相似的小群体的热点，ItemCF的推荐结果着重于维系用户的历史兴趣，因为推荐的是跟用户之前喜欢的物品的类似物品。  
UserCF的推荐更社会化，反映了用户所在小型兴趣群体种物品的热门程度， ItemCF的推荐更加的个性化，反应了用户自己的兴趣的传承。


特征粒度的问题：  
e.g.新闻网站，用户的兴趣都不可能特别的细化，因为一般不可能把兴趣的特征细化到喜欢某个话题这么小的粒度，因为话题与情景相关，也就是与当时社会发生的事件可能相关。但是用户每天都会有浏览新闻的习惯，所以只有新闻版面的粗粒度，例如喜欢看社会新闻或者体育新闻。  
个性化新闻推荐更加强调抓住新闻的热点，热门程度和时效性是个性化新闻推荐的重点，个性化针对这两点比较次要。  
所以这样应该是基于UserCF推荐，因为保证当天新闻的时效性，而且根据相似的用户行为保证了一定程度的个性化。  
从技术角度考虑，因为新闻更新速度快，Item的样本容量很大，并且更新很快，所以技术上很难实现，而UserCF只需要用用户相似表，而且用户更新速度远慢于新闻的更新速度。  

在图书、电子商务和电影网站：用ItemCF  
因为用户的兴趣比较固定和持久，所以应当根据用户的兴趣来设计。并且在这类网站当中，用户的数量一般很大，物品的数目相对较少，与新闻网站相对。


#### ItemCF相似度修改：
因为$ w_{ij} = \frac{|N(i)\cap N(j)|}{\sqrt{|N(i)||N(j)|}}$ 中，即使对N(j)做了修正，但是N(j)很大的时候，依然会有比较大的相似度。
另外一种相似度修改方法：
$$ w_{ij} = \frac{|N(i)\cap N(j)|}{|N(i)|^{1-\alpha}|N(j)|^{\alpha}} $$   
$\alpha = 0.5$是准确率和召回率最高，提高$\alpha$会降低准确率和召回率，但是会提高覆盖率和新颖性（降低流行度也就是提高新颖性）。
